# 1. Labraries Loading

In [344]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import MinMaxScaler

# 2. Dataset opening

In [345]:
measurementsDf = pd.read_json('../../../../Data/RawData/measurements.json')

In [346]:
timeDf = measurementsDf.pop('time')
timeDf = timeDf.to_frame()
sensors_id_Df = measurementsDf.pop('sensor_id')
sensors_id_Df = sensors_id_Df.to_frame()

In [347]:
measurementsDf.head(20)

,T,Rn,H,P,CO2
0,3.000000,NaN,14.000000,NaN,NaN
1,31.300000,NaN,44.475000,NaN,567.750000
2,31.333333,NaN,45.505556,NaN,636.361111
3,31.451429,NaN,47.145714,NaN,734.000000
4,31.161765,NaN,36.000000,NaN,461.529412
5,31.008333,NaN,28.436111,NaN,400.000000
6,30.647222,NaN,29.133333,NaN,400.000000
7,29.891429,NaN,28.597143,NaN,400.000000
8,29.118919,NaN,29.162162,NaN,400.000000
9,28.200000,NaN,31.611111,NaN,400.000000


In [321]:
discrete_features = [col for col in measurementsDf.columns if measurementsDf[col].dtype != object]

In [322]:
discrete_features

['T', 'Rn', 'H', 'P', 'CO2']

In [323]:
print(measurementsDf.shape)

(117274, 5)


# 3. Data Preprocessing

In [324]:
# Its important to check for the null values

measurementsDf.isnull().sum()

T      38122
Rn      3160
H      38106
P      30649
CO2     3851
dtype: int64

In [325]:
# Null values visualization
#measurementsDf = measurementsDf.drop(['time'],axis = 1)
measurementsDf[measurementsDf['T'].isnull()]

,T,Rn,H,P,CO2
1088,NaN,10.000000,NaN,NaN,NaN
1089,NaN,3691.898734,NaN,NaN,NaN
1228,NaN,NaN,NaN,101.689026,183.743590
1229,NaN,NaN,NaN,101.680974,181.868421
1230,NaN,NaN,NaN,101.692096,181.019231
...,...,...,...,...,...
117267,NaN,NaN,11.812500,NaN,NaN
117268,NaN,NaN,11.333333,NaN,NaN
117269,NaN,NaN,11.210526,NaN,NaN
117270,NaN,NaN,8.500000,NaN,NaN


## 3.1 Data inputation using mean, median and most frequent from sklearn input librabry

### Mean inputation

In [326]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,T,Rn,H,P,CO2
0,3.000000,648.532105,648.532105,648.532105,648.532105
1,31.300000,648.532105,648.532105,648.532105,648.532105
2,31.333333,648.532105,648.532105,648.532105,648.532105
3,31.451429,648.532105,648.532105,648.532105,648.532105
4,31.161765,648.532105,648.532105,648.532105,648.532105
5,31.008333,648.532105,648.532105,648.532105,648.532105
6,30.647222,648.532105,648.532105,648.532105,648.532105
7,29.891429,648.532105,648.532105,648.532105,648.532105
8,29.118919,648.532105,648.532105,648.532105,648.532105
9,28.200000,648.532105,648.532105,648.532105,648.532105


### Median inputation

In [331]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')

measurementsDf['T'] =  imputer.fit_transform(measurementsDf[["T"]]).ravel()
measurementsDf['Rn'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['H'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['P'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()
measurementsDf['CO2'] =  imputer.fit_transform(measurementsDf[["Rn"]]).ravel()

measurementsDf.head(15)

,T,Rn,H,P,CO2
0,3.000000,319.629167,319.629167,319.629167,319.629167
1,31.300000,319.629167,319.629167,319.629167,319.629167
2,31.333333,319.629167,319.629167,319.629167,319.629167
3,31.451429,319.629167,319.629167,319.629167,319.629167
4,31.161765,319.629167,319.629167,319.629167,319.629167
5,31.008333,319.629167,319.629167,319.629167,319.629167
6,30.647222,319.629167,319.629167,319.629167,319.629167
7,29.891429,319.629167,319.629167,319.629167,319.629167
8,29.118919,319.629167,319.629167,319.629167,319.629167
9,28.200000,319.629167,319.629167,319.629167,319.629167


### Iteratve imputation

In [336]:
iterative_imputater = IterativeImputer()

In [337]:
iter_imputer = IterativeImputer(random_state=42)
iter_imputed = iter_imputer.fit_transform(measurementsDf[discrete_features])
df_iter_imputed = pd.DataFrame(iter_imputed, columns=discrete_features)
#iterative_imputater.fit_transform(measurementsDf)

In [338]:
timeDf = timeDf.join(df_iter_imputed)
timeDf.join(sensors_id_Df)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,3.000000,2404.663876,14.000000,102.643690,150.232208,@0123
1,2020-07-14T14:00:00.000Z,31.300000,448.456100,44.475000,101.775235,567.750000,D0001
2,2020-07-14T15:00:00.000Z,31.333333,417.807923,45.505556,101.786788,636.361111,D0001
3,2020-07-14T16:00:00.000Z,31.451429,365.207804,47.145714,101.801547,734.000000,D0001
4,2020-07-14T17:00:00.000Z,31.161765,775.108082,36.000000,101.719291,461.529412,D0001
...,...,...,...,...,...,...,...
117269,2020-06-27T10:00:00.000Z,30.545190,1725.951292,11.210526,101.557905,88.493014,undefined
117270,2020-06-27T11:00:00.000Z,30.816691,1827.157380,8.500000,101.531039,87.653993,undefined
117271,2020-06-27T12:00:00.000Z,30.766608,1808.488296,9.000000,101.535995,87.808764,undefined
117272,2020-08-19T13:00:00.000Z,30.000000,262.134392,48.000000,101.800679,92.869113,undefined


In [339]:
timeDf.isna().sum()

time    0
T       0
Rn      0
H       0
P       0
CO2     0
dtype: int64

### KNN imputation

For the KNN imputation work as planned, we must first normalize the data using one of the scikit learn libraries, MinMaxScaler, which will transform the data to values beetween 0 and 1 

In [348]:
scaler = MinMaxScaler()
scaledDf = pd.DataFrame(scaler.fit_transform(measurementsDf), columns = measurementsDf.columns)

We can see bellow that the featueres have values between 0 and 1

In [349]:
scaledDf.head()

,T,Rn,H,P,CO2
0,0.070258,NaN,0.147368,NaN,NaN
1,0.733021,NaN,0.468158,NaN,0.047865
2,0.733802,NaN,0.479006,NaN,0.053650
3,0.736567,NaN,0.496271,NaN,0.061881
4,0.729784,NaN,0.378947,NaN,0.038910


Now everything is ready to realize the imputation

In [350]:
%%time

imputer = KNNImputer(n_neighbors=5)
scaledDf = pd.DataFrame(imputer.fit_transform(scaledDf),columns = scaledDf.columns)

CPU times: total: 10min 52s
Wall time: 8min 16s


In [351]:
scaledDf

,T,Rn,H,P,CO2
0,0.070258,0.002555,0.147368,0.932939,0.004434
1,0.733021,0.270933,0.468158,0.967662,0.047865
2,0.733802,0.159120,0.479006,0.968301,0.053650
3,0.736567,0.000027,0.496271,0.967408,0.061881
4,0.729784,0.282132,0.378947,0.967054,0.038910
...,...,...,...,...,...
117269,0.474948,0.038167,0.118006,0.968167,0.003072
117270,0.096974,0.004726,0.089474,0.967093,0.002088
117271,0.110997,0.003265,0.094737,0.966434,0.002352
117272,0.702576,0.020834,0.505263,0.970596,0.031111


In [352]:
scaledDf.isna().any()

T      False
Rn     False
H      False
P      False
CO2    False
dtype: bool

After the imputation is done, the dataframe values are reversed to the original state

In [353]:
reverseDf = scaler.inverse_transform(scaledDf)

In [354]:
pd.DataFrame(reverseDf).head(20)

,0,1,2,3,4
0,3.000000,9.442842,14.000000,97.921325,52.594737
1,31.300000,1001.448424,44.475000,101.565847,567.750000
2,31.333333,588.154974,45.505556,101.632854,636.361111
3,31.451429,0.100000,47.145714,101.539128,734.000000
4,31.161765,1042.843803,36.000000,101.501998,461.529412
5,31.008333,1585.713833,28.436111,101.567130,400.000000
6,30.647222,1585.713833,29.133333,101.567130,400.000000
7,29.891429,1585.713833,28.597143,101.567130,400.000000
8,29.118919,1585.713833,29.162162,101.567130,400.000000
9,28.200000,1585.713833,31.611111,101.567130,400.000000


In [355]:
#knn_imputed_df = pd.DataFrame(knn_imputer.transform(measurementsDf[discrete_features])).head(20)

Finally, the dropped non-numerical columns are put back

In [356]:
timeDf = timeDf.join(scaledDf)
timeDf.join(sensors_id_Df)

,time,T,Rn,H,P,CO2,sensor_id
0,2020-06-27T10:00:00.000Z,0.070258,0.002555,0.147368,0.932939,0.004434,@0123
1,2020-07-14T14:00:00.000Z,0.733021,0.270933,0.468158,0.967662,0.047865,D0001
2,2020-07-14T15:00:00.000Z,0.733802,0.159120,0.479006,0.968301,0.053650,D0001
3,2020-07-14T16:00:00.000Z,0.736567,0.000027,0.496271,0.967408,0.061881,D0001
4,2020-07-14T17:00:00.000Z,0.729784,0.282132,0.378947,0.967054,0.038910,D0001
...,...,...,...,...,...,...,...
117269,2020-06-27T10:00:00.000Z,0.474948,0.038167,0.118006,0.968167,0.003072,undefined
117270,2020-06-27T11:00:00.000Z,0.096974,0.004726,0.089474,0.967093,0.002088,undefined
117271,2020-06-27T12:00:00.000Z,0.110997,0.003265,0.094737,0.966434,0.002352,undefined
117272,2020-08-19T13:00:00.000Z,0.702576,0.020834,0.505263,0.970596,0.031111,undefined


In [357]:
timeDf.isna().sum()

time    0
T       0
Rn      0
H       0
P       0
CO2     0
dtype: int64

# Final dataframe visualization

In [358]:
measurementsDf.head(20)

,T,Rn,H,P,CO2
0,3.000000,NaN,14.000000,NaN,NaN
1,31.300000,NaN,44.475000,NaN,567.750000
2,31.333333,NaN,45.505556,NaN,636.361111
3,31.451429,NaN,47.145714,NaN,734.000000
4,31.161765,NaN,36.000000,NaN,461.529412
5,31.008333,NaN,28.436111,NaN,400.000000
6,30.647222,NaN,29.133333,NaN,400.000000
7,29.891429,NaN,28.597143,NaN,400.000000
8,29.118919,NaN,29.162162,NaN,400.000000
9,28.200000,NaN,31.611111,NaN,400.000000
